# ДЗ 2. Проверка гипотез

Имя, Фамилия: Игнат Сальников

группа: 221

**Оценка(для проверяющего):** 0 из 32

**Дедлайн:** 24 ноября 23:59 (решение нужно сдать в энитаск)

# Введение. Несколько слов о шахматах

В этом домашнем задании вам будет представлен набор данных связанных с шахматами. Напомним, что шахматная партия может заканчиваться одним из трёх исходов:
- Победа белых (записывается как 1-0)
- Победа чёрных (записывается как 0-1)
- Ничейный исход (записывается как 1/2-1/2)

Также, в соревновательных шахматах есть временной контроль: партии делятся на несколько категорий, в зависимости от того, сколько времени оппонненты имеют на размышления:
- Блиц (до 10 минут на все ходы, возможно с добавлением времени после каждого хода)
- Рапид (от 10 до 30 минут на все ходы, возможно с добавлением времени после каждого хода)
- Классические или медленные шахматы (от 30 минут на все ходы, часто с добавлением времени после 40 и 60 ходов)

Для того чтобы как-то классифицировать силу игроков, была введена рейтинговая система ELO, на основании которой игрокам присуждается ранг (например международный мастер или гроссмейстер) и формируются турниры. Отметим что рейтинг ELO меняется, в зависимости от того, в каком формате временного контроля игрок участвует (т.е. у одного человека разный рейтинг для блица, рапида и классических шахмат). 

В этом датасете собраны основные данные об шахматных партиях, сыгранными за период с 1980 по 2021 года.
- Представлены партии всех трех форматов, формат партии указан в поле ``control``
- Исход партии представлен в поле ``result``
- Рейтинги оппонентов записаны в полях ``elo_white`` и ``elo_black`` для рейтинга играющих за белых и за черных соответственно.  
    - Поле ``elo_average`` содержит среднее значение ``elo_white`` и ``elo_black`` -- _усредненный рейтинг (оппонентов)_. 
    - Поле ``elo_difference`` содержит в себе значение ``elo_white - elo_black``. 
- Поле ``opening`` содержит одно из пяти значений ``'A', 'B', 'C', 'D', 'E'``, которое кодирует информацию о нескольких первых ходах в партии (см. задачу 3 для деталей). 
- В поле ``length`` записано число сделанных ходов в партии. Обратите внимание, что ход -- это движение одной фигуры со стороны белых **и** движение одной фигуры со стороны чёрных, т.е. за один ход и белые и черные совершают по действию.
- Поля ``date`` и ``site`` содержат информацию о том, в каком году была сыграна партия и место её проведения соответственно. 

## Обзор данных (2 балла)
В этой секции вам предлагается познакомиться с данными. Посмотрите на три разных формата временного контроля. Сравните как в каждом из них 
- Распределены исходы партий (победа белых/победа чёрных/ничья)
- Распределены усреднённые рейтинги оппонентов партий 
- Распределены длительности партий 

**Бонус (max 3 дополнительных балла)** При желании, можно также посмотреть другие вещи --- например как менялось количество партий по декадам или пятилеткам. Попробуйте обнаружить и _объяснить_ интересные и статистически значимые феномены.  

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.precision', 5)
import scipy.stats as stats

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import plotly.graph_objects as go

In [ ]:
games_df = pd.read_csv("data/games.csv")
games_df.head()

In [ ]:
games_df.info(verbose = True)

In [ ]:
result_distrib = games_df.groupby(['control', 'result']).size().reset_index(name='count')

total_per_control = result_distrib.groupby('control')['count'].transform('sum')
result_distrib['distrib'] = (result_distrib['count'] / total_per_control) * 100

px.bar(
    result_distrib, 
    x='control', y='distrib', color='result', 
    title="Distribution of game\'s result by format",
    barmode='group', text='distrib'
).update_traces(
    texttemplate='%{text:.1f}%', textposition='outside'
).update_layout(
    yaxis=dict(ticksuffix='%')
).show(renderer='png')

Видно, что в разных временных форматах совершенно разное распределение исходов партий. Это указывает нам на то что надо завести три различных набора данных и работать с каждым по отдельности:

Постройте для каждого из контролей распределение усредненного рейтинга партий и длин партий.

In [ ]:
px.violin(
    games_df, 
    x='elo_average', color='control', facet_row='control', orientation='h',
    box=True,
    title="Distribution of game\'s averaged elo by format"
).update_yaxes(matches=None).show(renderer='png')

In [ ]:
px.histogram(
    games_df, 
    x='elo_average', color='control', facet_row='control',
    title="Distribution of game\'s averaged elo by format",
    opacity=0.75,
    nbins=500,
    width=800,
    height=800,
    range_x=[600, None]
).update_yaxes(
    matches=None
).show(renderer='png')

In [ ]:
px.violin(
    games_df, 
    x='length', color='control', facet_row='control', orientation='h',
    box=True,
    title="Distribution of game\'s lengths by format",
).update_yaxes(matches=None).show(renderer='png')

In [ ]:
px.histogram(
    games_df, 
    x='length', color='control', facet_row='control',
    title="Distribution of game\'s lengths by format",
    opacity=0.75,
    nbins=500,
    width=800,
    height=800,
    range_x=[0, 300]
).update_yaxes(matches=None).show(renderer='png')

## Бонус (должно было быть больше, но я забил)

In [ ]:
games_df["skill_lavel"] = np.where(games_df["elo_average"] > 1500, "high", "low")

px.histogram(
    games_df, 
    x='date', color='control', facet_row='control',
    facet_col='skill_lavel', facet_col_spacing=0.05,
    title="Number of games in time by format and skill lavel",
    opacity=0.75,
    width=800,
    height=800,
    nbins=16 * 4
).update_yaxes(
    matches=None
).show(renderer='png')

Количество партий стабильно росто. Бум в популярности блица и рапида случился примерно с начала ковида. При этом, как мне кажется, популярны именно быстрые форматы так как они более зрелищные. Пики количества игр на низком уровне я бы отнес к проведению каких-нибудь крупных любительских турниров или может быть появлению онлайн шахмат.

# Задачи

## Разминка. Проверка гипотез о средних (5 баллов)

### Взгляд в прошлое. Рейтинг игр в Праге. (2 балла)
Пусть $X_1, \ldots, X_n \overset{\text{i.i.d}}{\sim} X$ --- выборка из некоторого распределения. Вспомните как строится асимптотический доверительный интервал для $\mathbb{E}[X]$, и, используя асимптотический z-test проверьте на уровне значимости $\alpha = 0.05$ гипотезу о том что математическое ожидание рейтинга партии равно 2389:
$$
    H_0 : \mathbb{E}[X] = 2389 \\
    H_1 : \mathbb{E}[X] \neq 2389
$$
для игр, сыгранных в Праге за период 1980-1989 г.

При справедливости $H_0$ имеем:

$$z = \frac{\bar{X} - 2389}{\hat{\sigma} / \sqrt{n}}; \qquad z \sim_{n \to \infty} \mathcal{N}(0, 1)$$

Для двухстороннего теста смотрим лежит ли $z$ в интервале $(z_{\alpha/2}, z_{1 - \alpha/2})$ и если нет, то отвергаем гипотезу. Для правосторонего отвергаем если $z > z_{1 - \alpha}$, а для левостороннего если  $z < z_{\alpha}$.

In [ ]:
from numpy.typing import ArrayLike
from typing import Tuple, Optional, Literal

hypothesis_side = Literal["both", "left", "right"]


class StatTestResult:
    """
    Wrapper that carries information about performed statistical test. 
    
    Attributes
    ----------
    statistics : float
        value of the test's statistics.
    p_value : float
        obtained p-value for the test's statistics.
    significance : float
        significance level for the performed test
    critical_value : float
        critical value of the test's statistics for the given significance level 
    test_name : str
        name of the test
    null_name: str
        formulation of the null hypothesis
    alternative_name : str
        formulation of the alternative hypothesis
    verdict: 
        test's verdict
    """
    def __init__(self, statistics : float, p_value : float, significance : float, critical_value : float,\
                        test_name : str, null_name : str, alternative_name : str):
        self.statistics = statistics
        self.p_value = p_value
        self.significance = significance
        self.critical_value = critical_value
        self.name = test_name
        self.null = null_name
        self.alternative = alternative_name
        
        if self.p_value < self.significance:
            self.verdict = f"Reject H0 at significance level {significance}"
        else:
            self.verdict = f"Accept H0 at significance level {significance}"

    def __repr__(self):
        return f"""
        {self.name}. 
        H0: {self.null}
        H1: {self.alternative}
        ===================================
        Statistics value: {self.statistics}. Critical value: {self.critical_value}
        P-value: {self.p_value}, 
        Verdict: {self.verdict}
        """
    def __str__(self):
        return f"Statistics: {self.statistics}, P-value: {self.p_value}"

In [ ]:
import scipy.stats as sps

def z_test_one_sample(sample : ArrayLike, null_mean : float,
                      significance : float = 0.05, tail : hypothesis_side = 'both') -> StatTestResult:
    """
        Performs asymptotic z-test for the one sample
        
        Parameters
        ----------
        sample : ArrayLike
            sample on which z-test will be performed 
        null_mean : float
            value of mean under null hypothesis.
        significance : float
            significance level for the performed test
        tail : Literal["both", "left", "right"]
            alternative side
            
        Returns
        -------
        res : StatTestResult
            An object containing infomration about test results
    """
    sample_mean = np.mean(sample)
    sample_std = np.std(sample, ddof=1)
    n = len(sample)
    
    z_statistic = np.sqrt(n) * (sample_mean - null_mean) / sample_std 
    
    if tail == 'both':
        p_value = 2 * sps.norm.sf(np.abs(z_statistic))
    elif tail == 'right':
        p_value = sps.norm.sf(z_statistic)
    elif tail == 'left':
        p_value = sps.norm.cdf(z_statistic)
        
        
    critical_value = sps.norm.ppf(1 - significance / 2) if tail == 'both' \
        else (sps.norm.ppf(1 - significance) if tail == 'right' else sps.norm.ppf(significance))
    
    result = StatTestResult(
        statistics=z_statistic,
        p_value=p_value,
        significance=significance,
        critical_value=critical_value,
        test_name="z-test",
        null_name=f"mu = {null_mean}",
        alternative_name=f"mu {'!=' if tail == 'both' else ('>' if tail == 'right' else '<')} {null_mean}"
    )
    
    return result

In [ ]:
data = games_df[(games_df['site'] == 'Prague') & (games_df['date'] >= 1980) \
                & (games_df['date'] <= 1989)]['elo_average']

z_test_one_sample(data, 2389, tail='right')

**Заключение**: Не отвергаем гипотезу, что тут еще сказать 

### Назад в будущее. Рейтинг игр в Москве. (3 балла)
После того как вы проверили эту гипотезу, проверьте гипотезу о том, что средний рейтинг классических партий проводимых в Москве в 2010-2019 годах выше, чем рейтинг в 2000-2009 годах. Вспомните как устроен z-тест для двух выборок и 
1. сформулируйте нулевую гипотезу и альтернативу;
2. укажите статистику и её распределение при верной нулевой гипотезе;
3. реализуйте тест и проверьте гипотезу на уровне значимости 0.05;

$$
    H_0 : \mathbb{E}[X] = \mathbb{E}[Y] \\
    H_1 : \mathbb{E}[X] < \mathbb{E}[Y]
$$

По сути мы проверяем гипотезу о том не хуже ли рейтинг в 2010-2019 годах, но как мы увидим ниже этого достаточно.

При справедливости $H_0$ и независимости $X$ и $Y$ (как у нас) имеем:

$$z = \dfrac{\bar{X} - \bar{Y}}{\sqrt{\hat{\sigma}_1^2/n_1 + \hat{\sigma}_2^2/{n_2}}}; \qquad z \sim_{n \to \infty} \mathcal{N}(0, 1)$$


Для двухстороннего теста смотрим лежит ли $z$ в интервале $(z_{\alpha/2}, z_{1 - \alpha/2})$ и если нет, то отвергаем гипотезу. Для правосторонего отвергаем если $z > z_{1 - \alpha}$, а для левостороннего если  $z < z_{\alpha}$.

In [ ]:
def z_test_two_sample(sample_x : ArrayLike, sample_y : ArrayLike, null_mean_diff : float = 0,
                      significance : float = 0.05, tail : hypothesis_side = 'both') -> StatTestResult:
    """
        Performs asymptotic z-test for the two samples
        
        Parameters
        ----------
        sample_x : ArrayLike
            first sample for the z-test 
        sample_y : ArrayLike
            second sample for the z-test
        null_mean_diff : float
            differences of means under null hypothesis.
        significance : float
            significance level for the performed test
        tail : Literal["both", "left", "right"]
            alternative side
            
        Returns
        -------
        res : StatTestResult
            An object containing infomration about test results
    """
    
    sample_mean_x = np.mean(sample_x)
    sample_mean_y = np.mean(sample_y)
    
    n_x = len(sample_x)
    n_y = len(sample_y)
    
    std_of_sample = np.sqrt(np.std(sample_x, ddof=1)**2 / n_x + np.std(sample_y, ddof=1)**2 / n_y)
    
    z_statistic = (sample_mean_x - sample_mean_y - null_mean_diff) / std_of_sample 
    
    if tail == 'both':
        p_value = 2 * sps.norm.sf(np.abs(z_statistic))
    elif tail == 'right':
        p_value = sps.norm.sf(z_statistic)
    elif tail == 'left':
        p_value = sps.norm.cdf(z_statistic)
        
    critical_value = sps.norm.ppf(1 - significance / 2) if tail == 'both' \
        else (sps.norm.ppf(1 - significance) if tail == 'right' else sps.norm.ppf(significance))
    
    result = StatTestResult(
        statistics=z_statistic,
        p_value=p_value,
        significance=significance,
        critical_value=critical_value,
        test_name="two sample z-test",
        null_name=f"mu_x = mu_y",
        alternative_name=f"mu_x {'!=' if tail == 'both' else ('>' if tail == 'right' else '<')} mu_y"
    )
    
    return result

In [ ]:
data_x = games_df[(games_df['site'] == 'Moscow') & (games_df['date'] >= 2010) \
                & (games_df['date'] <= 2019) & (games_df['control'] == 'slow')]['elo_average']

data_y = games_df[(games_df['site'] == 'Moscow') & (games_df['date'] >= 2000) \
                & (games_df['date'] <= 2009) & (games_df['control'] == 'slow')]['elo_average']

z_test_two_sample(data_x, data_y, tail='left')

**Заключение**: Мы отвергли нулевую гипотезу - а значит в 2010-2019 годах рейтинг не был лучше.

## Задача 1. Инфляция рейтинга (5 баллов)

В течение времени правила подсчета рейтинга менялись. Для сравнения возьмем партии в классическом временном контроле и их рейтинги за периоды 2000-2009 года и 2010-2019 год. Нарисуйте гистограммы, которые описывают распределение усредненных рейтингов оппонентов в партиях за эти периоды.

In [ ]:
games_df_1 = games_df[(games_df['control'] == 'slow') & (games_df['date'] >= 2000) & (games_df['date'] <= 2019)].copy()
games_df_1['decade'] = np.where(games_df_1['date'] >= 2010, '2010-2019', '2000-2009')

px.histogram(
    games_df_1, 
    x='elo_average',
    barmode='overlay',
    color='decade',
    title="Distribution of game\'s averaged elo in slow format between in 2000-2009 and 2010-2019",
    width=900,
    nbins=600,
).show(renderer='png')

px.histogram(
    games_df_1, 
    x='elo_average',
    color='decade',
    facet_row='decade', 
    title="Distribution of game\'s averaged elo in slow format between in 2000-2009 and 2010-2019",
    width=900,
    height=800,
    nbins=600,
).show(renderer='png')

Распространено мнение, что в силу изменений правил подсчета, высокий рейтинг стало получить легче --- произошла инфляция рейтинга. Проверьте, так ли это на самом деле, с помощью рангового критерия [Уилкоксона-Манна-Уитни](http://www.machinelearning.ru/wiki/index.php?title=Критерий_Уилкоксона-Манна-Уитни). В качестве распределения статистики можете брать подходящую аппроксимацию нормальным распределением с поправкой на повторяющиеся значения. Как и ранее,
1. сформулируйте нулевую гипотезу и альтернативу; объясните смысл математической формулировки нулевой гипотезы;
2. укажите статистику и её распределение при верной нулевой гипотезе;
3. реализуйте тест и проверьте гипотезу на уровне значимости 0.05;

$X - \text{выборка 2010-2019 годов}, Y - \text{выборка 2000-2009 годов}$

$$
    H_0 : P[X > Y] = 1/2 \\
    H_1 : P[X < Y] > 1/2
$$

Смысл - проверяем что распределения одинаковаые, против альтернативы, что распределение $X$ смещено влево отностительно $Y$.

$R$ - ранги совместно упорядоченных наблюдений

$$
U_X =  n_1 n_2 + \frac{n_1 (n_1 + 1)}{2} - R_X \\ 
U_Y =   n_1 n_2 + \frac{n_2 (n_2 + 1)}{2} - R_Y \\
$$

Дальше в зависимости атальтернативы выбираем знаение $U$:
- $P(X > Y) > 1/2 \implies U = U_X$
- $P(Y > X) > 1/2 \implies U = U_Y$
- $P(X > Y) != 1/2 \implies U = \text{min}(U_X, U_Y)$

Запишем

$$z = \frac{U - \mu_U}{\sigma_U} ; \qquad z \sim_{n \to \infty} \mathcal{N}(0, 1)$$


Где

$$
\mu_U = \frac{n_1 n_2}{2}
$$

И в случае не абсолютно непрерывных распределений

$$
\sigma_U = \sqrt{\frac{n_1 n_2 (n_1 + n_2 + 1)}{12} - \frac{n_1 n_2 \sum_k (t_k^3 - t_k)}{12 \cdot (n_1 + n_2) \cdot (n_1 + n_2 - 1)}}; \qquad t_k - \text{количество дублей ранга } k \\
$$

Для правосторонего и левостороннего отвергаем если  $z < z_{\alpha}$, а для двухстороннего если  $z < z_{\alpha / 2}$.

In [ ]:
from scipy.stats import rankdata # this can help. See docs.

def mwu_test(sample_x : ArrayLike, sample_y : ArrayLike, significance : float = 0.05, tail : hypothesis_side = 'both') -> StatTestResult:
    """        
        Performs Mann–Whitney U-test with correct support for ties in the data. 
        REMARK: Test uses normal approximation for the distribution of test statistics
                under null hypothesis.
    
        Parameters
        ----------
        sample_x : ArrayLike
            first sample for the U-test 
        sample_y : ArrayLike
            second sample for the U-test
        significance : float
            significance level for the performed test
        tail : Literal["both", "left", "right"]
            alternative side
            
        Returns
        -------
        res : StatTestResult
            An object containing infomration about test results
    """
    
    combined_data = np.concatenate([sample_x, sample_y])
    ranks = rankdata(combined_data)
    
    n_x = len(sample_x)
    n_y = len(sample_y)
    
    ranks_x = ranks[:n_x]
    ranks_y = ranks[n_x:]
    
    nn = n_x * n_y
    n = n_x + n_y
    
    U_x = (nn + n_x * (n_x + 1) / 2 - np.sum(ranks_x))
    U_y = (nn + n_y * (n_y + 1) / 2 - np.sum(ranks_y))
            
    _, counts = np.unique(ranks, return_counts=True)
           
    sigma_ties = np.sqrt(nn / 12 * (n + 1 - np.sum(counts**3 - counts) / (n * (n - 1))))
    
    if tail == 'both':
                
        U = min(U_x, U_y)
        z = (U - nn / 2) /  sigma_ties
        p_value = 2 * sps.norm.cdf(z)
                
    elif tail == 'right':
        
        U = U_x
        z = (U - nn / 2) /  sigma_ties
        p_value = sps.norm.cdf(z)
                
    elif tail == 'left':
        
        U = U_y
        z = (U - nn / 2) /  sigma_ties
        p_value = sps.norm.cdf(z)
        
    critical_value = sps.norm.ppf(1 - significance / 2) if tail == 'both' \
        else (sps.norm.ppf(1 - significance) if tail == 'right' else sps.norm.ppf(significance))
    
    result = StatTestResult(
        statistics=U,
        p_value=p_value,
        significance=significance,
        critical_value=critical_value,
        test_name="U-test",
        null_name=f"P[x < y] = 1/2",
        alternative_name=f"P[x {'>' if tail == 'right' else '<'} y] {'!=' if tail == 'both' else '>'} 1/2"
    )
    
    print('Я возвращаю U статистику, которая используется для проверки гипотезу, а не как в scipy.\n')

    print(f"z = {z},    U_x = {U_x}    U_y = {U_y}")
    
    return result

In [ ]:
data_x = games_df[(games_df['date'] >= 2010) \
                & (games_df['date'] <= 2019) & (games_df['control'] == 'slow')]['elo_average']

data_y = games_df[(games_df['date'] >= 2000) \
                & (games_df['date'] <= 2009) & (games_df['control'] == 'slow')]['elo_average']

mwu_test(data_x, data_y, tail='left')

In [ ]:
sps.mannwhitneyu(data_x, data_y, alternative='less')

**Заключение**: Мы отвергли нулевую гипотезу - а значит в 2010-2019 годах рейтинг не был лучше и инфляции не произошло.

## Задача 2. Рейтинги и исход партий (7.5 баллов)

Исследуйте как, в зависимости от рейтинга участников, меняется распределение исхода партии. Мы предлагаем вам изучить как
- Разница в рейтинге влияет на шансы на результативный исход (победа одной из сторон)
- С ростом среднего рейтинга участников меняются шансы на победу

### Разрыв рейтинга и шансы на победу (5 баллов)

Нарисуйте гистограммы, показывающее как устроены исходы партий в зависимости от разницы в рейтинге для партий в разных формах временного контролля. 

In [ ]:
px.histogram(
    games_df, 
    x='elo_difference',
    color='result',
    facet_row='control', 
    barmode='overlay',
    title="Distribution of game\'s elo difference relatively to result in different formats",
    width=900,
    height=900,
    nbins=800,
    range_x=[-1200, 1200]
).update_yaxes(matches=None)

Проверьте гипотезу "симметрии":
> Правда ли что шансы победы белых при разрыве рейтинга между участниками в $X$ пунктов такие же, как шансы на победу чёрных при разрыве рейтинга в $-X$ пунктов?

Для проверки такой гипотезы будем использовать критерий $\chi^2$ для проверки независимости факторов. 
- Подумайте, как должна выглядеть факторная таблица в этом случае?
- Сформулируйте как интерпретировать нулевую гипотезу критерия $\chi^2$ в контексте проверяемой нами гипотезы.
- Реализуйте проверку гипотезы. 

Критерий однородности хи-хи для проверки независимости вероятностей победы от цвета. В таблице два столбца - результаты партий без нечьих, и две строки - знак разрыва рейтинга.

In [ ]:
games_df["elo_difference_sign"] = np.where(games_df["elo_difference"] > 0, "X > 0", "X < 0")
observed = pd.crosstab(games_df["elo_difference_sign"], games_df["result"]).drop(columns=['1/2-1/2'])
observed

В нашем контексте справедливость нулевой гипотезы будет значить, что шансы победы белых при разрыве рейтинга между участниками в $X$ пунктов такие же, как шансы на победу чёрных при разрыве рейтинга в $-X$ пунктов.

In [ ]:
def chi2_contingency(observed : ArrayLike, significance : float = 0.05) -> StatTestResult:
    """
        Performs chi-square independence test test for contingency table
        
        Parameters
        ----------
        observed : ArrayLike
            contingency tablefor the test 
        significance : float
            significance level for the performed test
            
        Returns
        -------
        res : StatTestResult
            An object containing infomration about test results
    """
    
    n = np.sum(observed)
    
    n_, m_ = observed.shape

    p_j = 1 / n * np.sum(observed, axis=1)
    q_k = 1 / n * np.sum(observed, axis=0)

    T = 0
    for j in range(n_):
        for k in range(m_):

            np_jq_k = n * p_j[j] * q_k[k]

            T += (observed[j][k] - np_jq_k)**2 / np_jq_k

    critical_value = sps.chi2.ppf(1 - significance, (n_ - 1) * (m_ - 1))
    
    p_value = sps.chi2.sf(T, (n_ - 1) * (m_ - 1))
    
    result = StatTestResult(
        statistics=T,
        p_value=p_value,
        significance=significance,
        critical_value=critical_value,
        test_name="Chi2-test",
        null_name=f"p_1 = p_2 = ... = p_k",
        alternative_name="not H0"
    )
    
    return result

Проверьте сформулированную выше гипотезу для ваших данных:

In [ ]:
chi2_contingency(observed.to_numpy())

**Заключение**: Ну очев, любой шахматист знает что у белых преимущество, даже по графикам видно что зеленая гистограмма выше, чем красная.

### Средний рейтинг и распределение исходов (2.5 балла)
Существует распространеное мнение, что чем выше рейтинг участников партии, тем более вероятно, что итогом партии станет ничья. Постройте гистограммы показывающее как устроены исходы партий в зависимости от рейтинга партии (``elo_average``) для партий в разных формах временного контролля. 

In [ ]:
px.histogram(
    games_df, 
    x='elo_average',
    color='result',
    facet_row='control', 
    barmode='overlay',
    title="Distribution of game\'s averaged elo relatively to result in different formats",
    width=900,
    height=900,
    nbins=300
).update_yaxes(matches=None).show(renderer='png')

Согласно ChessBase, игроков можно условно делить на следующие категории, в зависимости от их рейтинга:
- 0-1000 Begginer
- 1000-1600 Average club player level
- 1600-2100 Strong club player level
- 2100-2300 International league player
- 2300-2450 International Master (IM) level
- 2450-2650 Grandmaster (GM) level
- \> 2650 Supergrandmaster, world champion level

Вместо игроков, мы будем брать усредненный рейтинг оппонентов в партии и применять классификацию выше. С помощью статистических методов, проверьте влияет ли класс партии на распределение исходов. Используйте уже знакомый вам тест $\chi^2$.
- Подумайте, как должна выглядеть факторная таблица в этом случае?
- Сформулируйте как интерпретировать нулевую гипотезу критерия $\chi^2$ в контексте проверяемой нами гипотезы.
- Реализуйте проверку гипотезы. 

Таблица выгдядит так

In [ ]:
bins   = [0, 1000, 1600, 2100, 2300, 2450, 2650, 3000]
labels = ['Begginer', 'Average', 'Strong', 'International', 'IM', 'GM', 'SuperGM']

def get(df): 
    df = df.copy()
    df['category'] = pd.cut(df["elo_average"], bins=bins, labels=labels)
    observed = pd.crosstab(df["category"], df["result"])
    return observed

get(games_df)

В нашем контексте справедливость нулевой гипотезы будет значить, что уровень скилла игроков не влияет на распределение исходов.

In [ ]:
table_blitz = get(games_df[games_df['control'] == 'blitz'])
table_blitz

In [ ]:
# мы не можем иметь слишком маленькие бакеты для этого статтеста, так что дропнем Begginer для простоты
# можно было бы объединить его с Average, но мне лень 

print('Testing ... for slow format')
chi2_contingency(table_blitz.drop('Begginer').to_numpy())

In [ ]:
table_rapid = get(games_df[games_df['control'] == 'rapid'])
table_rapid

In [ ]:
# мы не можем иметь слишком маленькие бакеты для этого статтеста, так что дропнем Begginer для простоты

print('Testing ... for rapid format')
chi2_contingency(table_rapid.drop('Begginer').to_numpy())

In [ ]:
table_slow = get(games_df[games_df['control'] == 'slow'])
table_slow

In [ ]:
print('Testing ... for slow format')
chi2_contingency(table_slow.to_numpy())

**Заключение**: Мы видим, что распределение вероятностей исходов партий зависят от уровня скилла игроков.

Проблема в том что критерий $\chi^2$ позволяет нам проверять факторы на зависимость/независимость, но ничего не говорит о структуре этой зависимости. Реализуйте вычисление ранговой корреляции Спирмена и посчитайте её, чтобы оценить есть ли 
- Монотонная зависимость между *шансами на ничью* и *рангом партии*
- Монотонная зависимость между *разницей шансов на победу у белых и у чёрных* и *рангом партии*

Используйте для этого партии сыгранные в блиц-контроле и дайте интерпретацию полученным результатам. 

In [ ]:
table_blitz['draw_prob'] = table_blitz['1/2-1/2'] / table_blitz.sum(axis=1)
table_blitz['white_win_prob'] = table_blitz['1-0'] / table_blitz.sum(axis=1)
table_blitz['black_win_prob'] =  table_blitz['0-1'] / table_blitz.sum(axis=1)
table_blitz['dominance'] = table_blitz['white_win_prob'] - table_blitz['black_win_prob']

table_blitz

In [ ]:
def spearman_rank_correlation(observations : ArrayLike) -> float:
    """        
        Calculates spearmean's rank correlation coefficient for data in the format
            (1, X[1]), (2, X[2]), .... (n, X[n])
        i.e. for the paired rank data sorted by the first component. No ties assumed
        in the data X.
        
        Parameters
        ----------
        observed : ArrayLike
             array of values X[1], X[2], ... X[n], 
            
        Returns
        -------
        spearman : float 
            value of the spearman correlation for the given data
    """
    n = len(observations)
    
    ranks_x = np.arange(1, n + 1)
    ranks_y = rankdata(observations)
    
    centred_x = ranks_x - np.mean(ranks_x)
    centred_y = ranks_y - np.mean(ranks_y)

    rho = np.sum(centred_x * centred_y) / np.sqrt(np.sum(centred_x**2) * np.sum(centred_y**2))

    return rho

In [ ]:
rho = spearman_rank_correlation(table_blitz['draw_prob'].to_numpy())
print(f'Correlation between draw odds and game rank: {rho}')

In [ ]:
rho = spearman_rank_correlation(table_blitz['dominance'].to_numpy())
print(f'Correlation between white dominance over black and game rank: {rho}')

**Заключение.** Существует строгая монотонная зависимость между уровнем игры и вероятностью ничьи. Между уровнем игры и вероятностью побуды белых существует монотонная зависимость, но она столь выражена как для ничьих.

## Задача 3. Статистика дебютов (2.5 балла)

Другим распространённым мнением является то что выбор дебюта на высоком уровне не влияет на результативность партии. Все дебюты, согласно энциклопедии шахматных дебютов (ECO) делятся на 5 категорий
- A: Фланговые варианты.
- B: Полуоткрыте варианты.
- C: Открытые варианты.
- D: Закрытые и полузакрытые варианты.
- E: Системы типа индийской защиты.

Так как у нас нет доступа к партиям ИИ, которые сейчас считаются эталоном точной игры, мы ограничимся данными по партиям уровня супергроссмейстеров. 
Используя статистические методы, проверьте:
- Есть ли разница в предпочтениях дебютов (с точки зрения приведенных выше категорий) между партиями уровня супергроссмейстеров и партиями уровня гроссмейстеров в классическом временном контроле? 
- Правда ли что выбор между открытым (C) и закрытым (D) началом не влияет на распределение исхода партии? Проверьте это для разных форм временного контроля. Используйте партии только уровня супергроссмейстеров.

Для проверки будем использовать критерий $\chi^2$ (снова). Для каждой гипотезы, которое вы собираетесь проверять:
- сделайте визуализацию соответствующих данных;
- сформулируйте нулевую и альтернативную гипотезу. Опишите как применить критерий $\chi^2$ для проверки этих гипотез;
- реализуйте проверку гипотезы и дайте интерпретацию результатам;

Ну снова гипотеза что строки таблицы ниже однородны. Делаем табличку и закидываем в статтест.

In [ ]:
bins   = [0, 1000, 1600, 2100, 2300, 2450, 2650, 3000]
labels = ['Begginer', 'Average', 'Strong', 'International', 'IM', 'GM', 'SuperGM']

games_df['category'] = pd.cut(games_df["elo_average"], bins=bins, labels=labels)

px.histogram(
    games_df[(games_df['control'] == 'slow') & (games_df['category'].isin(['GM', 'SuperGM']))],
    x='opening', facet_row='category', 
    color='opening',
    title="Preferences of game\'s openings with GMs and SuperGMs"
).update_yaxes(matches=None).show(renderer='png')

In [ ]:
fter = (games_df['control'] == "slow")
observed = pd.crosstab(games_df[fter]["category"], games_df[fter]["opening"]).loc[['GM', 'SuperGM']]
observed

In [ ]:
chi2_contingency(observed.to_numpy())

**Заключение**: есть разница в предпочтениях дебютов между партиями уровня супергроссмейстеров и партиями уровня гроссмейстеров в классическом временном контроле.

Теперь проверьте влияет ли выбор между открытым и закрытым началом:

И опять гипотеза что строки таблицы ниже однородны. Делаем табличку и закидываем в статтест.

In [ ]:
px.histogram(
    games_df[(games_df['category'] == 'SuperGM') & (games_df['opening'].isin(['C', 'D']))], 
    x='opening',
    color='result', barmode='group',
    facet_col='control', 
    title="Game\'s openings with SuperGMs and game's results",
).update_yaxes(matches=None).show(renderer='png')

In [ ]:
fter = (games_df['control'] == "slow")
observed = pd.crosstab(games_df[fter]["opening"], games_df[fter]["result"]).loc[['C', 'D']]
observed

In [ ]:
chi2_contingency(observed.to_numpy())

In [ ]:
fter = (games_df['control'] == "rapid")
observed = pd.crosstab(games_df[fter]["opening"], games_df[fter]["result"]).loc[['C', 'D']]
observed

In [ ]:
chi2_contingency(observed.to_numpy())

In [ ]:
fter = (games_df['control'] == "blitz")
observed = pd.crosstab(games_df[fter]["opening"], games_df[fter]["result"]).loc[['C', 'D']]
observed

In [ ]:
chi2_contingency(observed.to_numpy())

Что изменится если брать уровень значимости 0.01? А если брать уровень значимости 0.1? Дайте ответы на эти вопросы и напишите интерпретацию полученных результатов.

In [ ]:
fter = (games_df['control'] == "blitz")
observed = pd.crosstab(games_df[fter]["opening"], games_df[fter]["result"]).loc[['C', 'D']]
chi2_contingency(observed.to_numpy(), significance=0.1)

In [ ]:
fter = (games_df['control'] == "rapid")
observed = pd.crosstab(games_df[fter]["opening"], games_df[fter]["result"]).loc[['C', 'D']]
chi2_contingency(observed.to_numpy(), significance=0.01)

In [ ]:
fter = (games_df['control'] == "slow")
observed = pd.crosstab(games_df[fter]["opening"], games_df[fter]["result"]).loc[['C', 'D']]
chi2_contingency(observed.to_numpy(), significance=0.01)

**Заключение**: выбор между открытым (C) и закрытым (D) началом зтатзначимо (p-value > 0.05) влияет на распределение исхода партии только для классическийх шахмат и рапида. Для блитца выбор дебюта играет меньшую роль так как там сильно сложнее реализовать стратегическое преимущество. 

Изменение уровня значимости не повлияло на результаты - у нас много данных и тест вышел достаточно мощный.

## Задача 4. Исследование длин партий в разных формах временного контроля (10 баллов)

Попробуйте разобраться в том, какому распределению следуют длительности партий в формате рапид. Выберете несколько распределений и попробуйте оценить их параметры. Постройте гистограммы на которых изображено распределение длительности партий и теоретические плотности распределений с оценёнными параметрами. 

Для оценки параметров распределений можно использовать метод ``fit()`` класса ``scipy.rv_continious()``.

In [ ]:
data = games_df[games_df['control'] == 'rapid']['length']
fig = px.histogram(data, x='length', opacity=0.2, nbins=300, histnorm='probability density')

fig.update_layout(title="Imperical pdf")
fig.show(renderer='png')

Типикл гамма распределение (лол, я был не прав), но го чекнем еще и другие

In [ ]:
from scipy.stats import gamma, lognorm, exponnorm, invgauss, invgamma, rayleigh, maxwell

distributions = {
    "Gamma": gamma,
    "LogNormal": lognorm,
    "ExponNorm": exponnorm,
    "InvGauss": invgauss,
    "InvGamma": invgamma,
    "Rayleigh": rayleigh,
    "Maxwell": maxwell
}

x_values = np.linspace(data.min(), data.max(), len(data))

for name, dist in distributions.items():
    
    params = dist.fit(data)
    pdf = dist.pdf(x_values, *params)
    fig.add_trace(go.Scatter(x=x_values, y=pdf, mode='lines', name=name))

fig.update_layout(title="Imperical pdf and fitted distrubutions")
fig.show()

По итогу отобрал ExponNormal, LonNormal, и InvGamma (которое совпадает с LonNormal на зафиченных параметрах).

In [ ]:
fig = px.histogram(data, x='length', opacity=0.2, nbins=300, histnorm='probability density')

distributions = {
#     "Gamma": gamma,
    "LogNormal": lognorm,
    "ExponNorm": exponnorm,
#     "InvGauss": invgauss,
    "InvGamma": invgamma,
#     "Rayleigh": rayleigh,
#     "Maxwell": maxwell
}

for name, dist in distributions.items():
    
    params = dist.fit(data)
    pdf = dist.pdf(x_values, *params)
    fig.add_trace(go.Scatter(x=x_values, y=pdf, mode='lines', name=name))


fig.update_layout(title="Imperical pdf and fitted distrubutions")
fig.show(renderer='png')

Выберите три наиболее на ваш взгляд подходящих распределения и постройте для них Q-Q график. Подробнее про Q-Q графики можно прочитать [здесь](https://habr.com/ru/articles/578754/) (обратите внимание, что в статье делается упор на Q-Q графики относительно нормального распределения --- вам же нужно построить графики относительно распределений, параметры которых вы оценили).

In [ ]:
def qqplot(data : ArrayLike, distribution: stats.rv_continuous,  quantiles : int,  ax : plt.axis) -> plt.axis:
    """
        Plots Q-Q plot agains theoretical distribution. 
        
        Parameters
        ----------
        data : ArrayLike
            sample data
        distribution: stats.rv_continuous
            theoretical distribution agains which quantiles will be plotted
        quantiles: int
            number of quantiles (must be less than size of the data)
        ax: plt.axis
            PyPlot Axis object, on which QQ-plot should be plotted
            
        Returns
        -------
        ax: plt.axis 
            PyPlot Axis object with QQ-plot
    """
    
    params = dist.fit(data)
    
    theoretical_quantiles = np.linspace(0, 1, quantiles + 2)[1:-1]
    theoretical_values = distribution.ppf(theoretical_quantiles, *params)

    sample_quantiles = np.percentile(data, theoretical_quantiles * 100)

    ax.scatter(theoretical_values, sample_quantiles, label="quantiles")
    ax.plot(theoretical_values, theoretical_values, color='red', linestyle='--', label='best fit')
    ax.grid()
    ax.legend()
    
    return ax

In [ ]:
fig, axs = plt.subplots(1, 3, figsize = (15, 5))

distributions = {
#     "Gamma": gamma,
    "LogNormal": lognorm,
    "ExponNorm": exponnorm,
#     "InvGauss": invgauss,
    "InvGamma": invgamma,
#     "Rayleigh": rayleigh,
#     "Maxwell": maxwell
}

for i, (name, dist) in enumerate(distributions.items()):
    qqplot(data, distribution=dist, quantiles=100, ax=axs[i])
    axs[i].set_title(name)

plt.show()

Ммм, три одинаковых графика, очень полезно. По грфику с pdf мне кажется что это эксп-нормальное.

 На основе всего выше сделанного выберете одно распределение, которое вам кажется наиболее подходящим. После того как вы выбрали распределение $F_0$, проверьте соответствующую гипотезу согласия. Конкретнее, проверьте гипотезу
$$
H_0: X_1,..,X_n \sim F_0
$$
против альтернативы
$$
H_A: X_1,..,X_n \nsim F_0
$$


Такую гипотезу часто проверяют с помощью статистики критерия Колмогорова-Смирнова:
$$
D = \sqrt{n} \sup_x \vert F_{0} - \hat{F}(x)\vert,
$$
где $\hat{F}$ -- эмпирическая функция распределения, которая задётся как
$$
\hat{F}(x) = \frac{1}{n}\sum_{i=1}^n \mathbb{1}(X_i \leq x).
$$
Выражение $1(условие)$ равно $1$, если условие верно и $0$ в противном случае -- так выше считается количество элементов выборки $\leq x$. Статистика $D$ при больших $n$ (сотен уже достаточно) имеет распределение Колмогорова, так что можно построить критерий для проверки. Чтобы упростить техническую часть с подсчётом $D$, воспользуйтесь готовым тестом из пакета scipy.stats.

In [ ]:
stats.kstest(data, 'exponnorm', exponnorm.fit(data))

Объясните, почему получилось такое значение несмотря на то, что подобранное вами распределение достаточно неплохо описывает данные? Подумайте, можно ли как-то изменить процедуру проверки для того чтобы можно было воспользоваться этим тестом? Обратите внимание на две вещи:
1. Как считается статистка критерия КС и почему для наших данных эмпирическая функция распределения никогда не будет сходится к функции распределения выбранного вами распределения? Как можно сделать дискретные данные  не-дискретными?
2.  Обратите внимание на выброс в данных в районе где достигается значение статистики критерия (statistic location). С чем может быть связан этот выброс и как можно было бы его устранить?

Опишите и реализуйте измененную процедуру. 

Этот критерий очень строгий, а у нас дискретное распределение, значит $\vert F_{0} - \hat{F}(x)\vert$ не сходится к нуля с ростом $n$ даже для вроде как одинаковых распределений. К тому же у нас есть выброс в районе где достигается значение статистики критерия (statistic location), что сильно ужудшает результаты теста. 

Для решения проблемы с дискретностью респределения добавим к нему случайный гаусовский шум с нулевым средним.

Методологически сложно выкинуть 'плотные' точки из распределения так, чтобы не оверфитнуться, но другие способы не работаю так что будем делать так - а именно разобъем данные из интересующей нас области на бакеты $X_i$ для эмперического распределения $Y_i$ для таргет распределения и будем урезать бакеты $X_i$ до размеров бакетов $Y_i$

In [ ]:
data = games_df[games_df['control'] == 'rapid']['length']

data_with_noize = data.to_numpy() + np.random.uniform(0, 1, len(data))
stats.kstest(data_with_soize, 'exponnorm', exponnorm.fit(data_with_soize))

In [ ]:
target = exponnorm.rvs(*exponnorm.fit(data_with_soize), size=len(data_with_soize))

def adjust_buckets(X, Y, n):
    X = np.sort(X.copy())
    Y = np.sort(Y.copy())

    X_bins = np.array_split(X, n)
    Y_bins = np.array_split(Y, n) 
    
    X_out = []

    for i in range(n):        
        X_out.extend(
            np.random.choice(X_bins[i], size=min(len(X_bins[i]), len(Y_bins[i])), replace=False)
        )
    return X_out

mask_X = (data_with_noize > 65) & (data_with_noize < 80)
mask_Y = (target > 65) & (target < 80)

X_out = adjust_buckets(data_with_noize[mask_X], target[mask_Y], n=800)

In [ ]:
data_with_noize = data_with_noize[~mask_X].tolist() + X_out

In [ ]:
stats.kstest(data_with_noize, 'exponnorm', exponnorm.fit(data_with_soize))

In [ ]:
fig = px.histogram(x=data_with_noize, opacity=0.2, nbins=300, histnorm='probability density')

params = exponnorm.fit(data_with_noize)
pdf = exponnorm.pdf(x_values, *params)
fig.add_trace(go.Scatter(x=x_values, y=pdf, mode='lines', name="ExponNorm"))

fig.update_layout(title="Imperical pdf and fitted distrubution")
fig.show(renderer='png')

Все равно не работает, а большим макакингом заниматься я не хочу
